In [8]:
import sys
!{sys.executable} -m pip install requests websocket-client

In [9]:
import requests

# No auth since you passed token=''
url = 'http://localhost:8888'

# Start a kernel
resp = requests.post(f'{url}/api/kernels')
kernel_id = resp.json()['id']
print("Kernel ID:", kernel_id)

Kernel ID: c8a38a82-fb6f-4c0b-8f93-452462aabbe3


In [12]:
import websocket
import json
import uuid

def send_execute_request(ws, code, kernel_id):
    msg_id = str(uuid.uuid4())
    msg = {
        "header": {
            "msg_id": msg_id,
            "username": "",
            "session": str(uuid.uuid4()),
            "msg_type": "execute_request",
            "version": "5.3"
        },
        "parent_header": {},
        "metadata": {},
        "content": {
            "code": code,
            "silent": False,
            "store_history": True
        }
    }
    ws.send(json.dumps(msg))
    return msg_id

In [14]:

# Connect to websocket
ws_url = f"ws://localhost:8888/api/kernels/{kernel_id}/channels"
ws = websocket.create_connection(ws_url)

# Send code to execute
msg_id = send_execute_request(ws, "2 + 2", kernel_id)

# Receive output
while True:
    msg = json.loads(ws.recv())
    if msg.get("parent_header", {}).get("msg_id") == msg_id:
        if msg["msg_type"] == "execute_result":
            print("Output:", msg["content"]["data"]["text/plain"])
            break

Output: 4
